In [13]:
import hashlib
import hmac
import datetime
import time

def generate_totp(secret_key):
    time_interval = 60
    current_time = time.time()
    time_counter = int(current_time / time_interval)
    time_counter_bytes = time_counter.to_bytes(8, byteorder='big')

    hmac_result = hmac.new(secret_key.encode(), time_counter_bytes, hashlib.sha1).digest()

    offset = hmac_result[-1] & 0x0F

    truncated_hash = hmac_result[offset:offset+4]

    truncated_hash_int = int.from_bytes(truncated_hash, byteorder='big')

    totp = truncated_hash_int % (10 ** 6)

    return '{:06d}'.format(totp)

SECRET_KEY = "ASOFTANDSWEETSENSATIONSOFLUFFY"

otps = []
for i in range(5):
    otp = generate_totp(SECRET_KEY)
    print(f"Generated OTP {i+1}: {otp}")
    otps.append(otp)
    if i < 4:
        time.sleep(60)

print("Generated OTPs:", otps)


Generated OTP 1: 670200
Generated OTP 2: 896920
Generated OTP 3: 689268
Generated OTP 4: 427323
Generated OTP 5: 587034
Generated OTPs: ['670200', '896920', '689268', '427323', '587034']


In [35]:
import json
import os
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives import padding

# User data
user_data = {
    "user_id": 12345,
    "vehicle_id": 67890
}
user_data_json = json.dumps(user_data).encode()

print("User Data (Before Encryption):", user_data)

# Encrypt Data
def encrypt_data(data, key):
    iv = os.urandom(16)
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv))
    encryptor = cipher.encryptor()

    padder = padding.PKCS7(128).padder()
    padded_data = padder.update(data) + padder.finalize()

    encrypted_data = encryptor.update(padded_data) + encryptor.finalize()
    return iv + encrypted_data

# Generate a 32-byte key from the fifth OTP
key = otps[4].ljust(32)[:32].encode()

encrypted_data = encrypt_data(user_data_json, key)
print("Encrypted Data (hex):", encrypted_data.hex())


User Data (Before Encryption): {'user_id': 12345, 'vehicle_id': 67890}
Encrypted Data (hex): 166a68a793c25f6988c3b443a88b3fba3ee6a2adb93ae5dce25372db939b8e1708febc4b05dc8a34c29be7baf9139736850b3a767b8e75f03195949a497608f7


In [36]:
import qrcode

def create_qr_code(data, filename):
    qr = qrcode.QRCode(
        version=1,
        error_correction=qrcode.constants.ERROR_CORRECT_L,
        box_size=10,
        border=4,
    )
    qr.add_data(data)
    qr.make(fit=True)

    img = qr.make_image(fill='black', back_color='white')
    img.save(filename)

create_qr_code(encrypted_data.hex(), 'encrypted_data_qr.png')


In [37]:
def decrypt_data(encrypted_data, key):
    iv = encrypted_data[:16]  # Extract IV from the beginning
    encrypted_data = encrypted_data[16:]  # The rest is the encrypted data

    cipher = Cipher(algorithms.AES(key), modes.CBC(iv))
    decryptor = cipher.decryptor()

    decrypted_padded_data = decryptor.update(encrypted_data) + decryptor.finalize()

    # Unpadding data
    unpadder = padding.PKCS7(128).unpadder()
    decrypted_data = unpadder.update(decrypted_padded_data) + unpadder.finalize()

    return decrypted_data.decode()

decrypted_data = decrypt_data(bytes.fromhex(encrypted_data.hex()), key)
print("Decrypted Data:", decrypted_data)

Decrypted Data: {"user_id": 12345, "vehicle_id": 67890}


In [38]:
import hashlib
import json
import pickle

def compute_sha3_hash(data):
    data_str = str(data)
    data_str = data_str.replace('"', "'")
    return hashlib.sha3_256(data_str.encode()).hexdigest()

# Compute SHA-3 hashes for original and decrypted data
original_hash = compute_sha3_hash(user_data)
decrypted_hash = compute_sha3_hash(decrypted_data)

print('Original Hash:', original_hash)
print('Decrypted Hash:', decrypted_hash)

if original_hash == decrypted_hash:
    print('Data Integrity Verified.')
else:
    print('Data Integrity Verification Failed.')


User Data: {'user_id': 12345, 'vehicle_id': 67890}
Decrypted Data: {"user_id": 12345, "vehicle_id": 67890}
Original Hash: 5cfaf0c1200c7453c9e29abc7ee4cc4b9a98049c207cba6bb18dbe5cf08b3865
Decrypted Hash: 5cfaf0c1200c7453c9e29abc7ee4cc4b9a98049c207cba6bb18dbe5cf08b3865
Data Integrity Verified.
